In [408]:
import pandas as pd
from pprint import pprint
df = pd.read_csv("pericias_medicas.csv", sep=",", encoding="utf-8")
df = df['text'].apply(str)
df1 = df.head(20)

In [335]:
# import re
# def general(txt: str, bert=False, nums=False) -> str:
#     """
#     Elimina caracteres no deseados
#     Params:
#         **txt**:texto a ser limpiado de caracteres no desaeados
#     """
#     if nums:
#         txt = re.sub(r'\d+', ' ', txt)
#     if not bert:
#         txt = txt.translate(str.maketrans(
#             'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))
#         txt = txt.lower()
        


#     txt = re.sub(' +', ' ', txt)
#     #txt = re.sub(r'[^\w\s]', '', txt)
#     txt = re.sub(r'\n\n+', '', txt)
#     txt = re.sub(r' \n \n', '', txt)
#     txt = re.sub(r'\. \n+', '\.\n', txt)
#     txt = re.sub(r' \n+', '\n', txt)
#     txt = re.sub(r'\d+', '', txt)
#     txt = re.sub(r'[/@ -"”“]', ' ', txt)
#     txt = re.sub(r' +(i{1,3}|iv|vi+|v|ix)', '', txt)
#     txt = re.sub(r'[A-Za-z-]+@[A-Za-z-]+.com', ' ', txt)
#     txt = re.sub(r'\\', '', txt)
    

    
#     return txt

# dfLimpio = df1.apply(general)

In [435]:
import re

def limpiarTexto(text):
    text = re.sub(r'[-,;+*?¿!¡<>{}// %&´""]',' ',text)
    text = re.sub('^ +',' ',text)
    text = re.sub('\n ','',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' +\n','\n',text)
    text = re.sub('\. \n','\.\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('¨[... ]','',text)
    text = text.lower()
    text = re.sub(' {2,100}',' ',text)
    text = re.sub('^ +','',text)

    
    
    return text

dfLimpio = df1.apply(limpiarTexto)

In [429]:
pprint(dfLimpio[0])

('\n'
 'informa a continuación:\n'
 'j.t. 1perito medico presenta informe pericial\n'
 'autos: “boiero franco ivan c swiss medical art s.a. s accidente ley '
 'especial”\n'
 'expediente n.º: 71399 201i proemio:\n'
 'sr. juez nacional:\n'
 'hortas maria andrea s. perito médico de oficio en los siguientes\n'
 'autos con domicilio constituido en la calle austria 1754 3º piso dpto. 10 de '
 'esta\n'
 'capital z 172 tel. 4325 8802 domicilio electrónico mashortas@hotmail.com '
 '216523394 3 en cumplimiento lo dispuesto por v.s. referente a los presentes '
 'autos\n'
 'y después de haber examinado a quién dijo ser boiero franco iván quién se\n'
 'tomaron en el momento del examen el estado clínico actual se consideraron '
 'la\n'
 'capacidad genérica y en particular los puntos de pericia solicitados por las '
 'partes\n'
 'y o v.s.:\n'
 'ii antecedentes de interes medico legal:\n'
 'foja 3 inicia la demanda\n'
 'foja 6 vta. hechos accidente in itinere refiere que el 20 de setiembre del '
 '201

## Limpieza de texto

In [208]:
import nltk

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

df1.apply(str)
df1 = df1.apply(remove_stops)


In [ ]:
#pprint(df1[10])

## Expresion regular para encontrar los titulos

In [445]:
#creo un nuevo data frame de titulos
def buscarTitulos(text):
    text = re.sub(r'^ ix|iv|vi+|i+x |(i{2,3})','', text)
    titulosBuscados = re.findall(r' [a-z -\.]+:\n', text)
    return titulosBuscados

def limpiarTitulos(text):
    text = re.sub(r'^ ','', text)
    text = re.findall(r'[a-z -]+:', text)
    #text = re.sub(r'\\n','', text)
    return text


dfTitulos = dfLimpio.apply(buscarTitulos)
dfTitulos = dfTitulos.apply(str)
dfTitulos = dfTitulos.apply(limpiarTitulos)
pprint(dfTitulos[0])
#HACER ESTADISTICAS DE TITULOS


[' proemio:',
 ' juez nacional:',
 ' antecedentes de interes medico legal:',
 ' examen fisico del actor:',
 ' semiologia de las afecciones:',
 ' tobillo:',
 ' consideraciones medico legales:',
 ' contestacion de los puntos de pericia:']


In [401]:
#lista de titulos y ubicacion
titulosPosicion=[]
for i,j in enumerate(titulos1):
    inicio = df1[0].index(titulos1[i])
    final = inicio + (len(titulos1[i]))
    palabrasPorTitulo = final-inicio
    titulosPosicion.append((titulos1[i], inicio,final, palabrasPorTitulo))
pprint(titulosPosicion)
#anotar parrafos entre titulos.
#parrafo = texto[inicio:inicio+1]

TypeError: list indices must be integers or slices, not str